# Keras 迁移学习实战

## 核心概念

迁移学习（Transfer Learning）将在大规模数据集上预训练的模型知识迁移到新任务，尤其适合数据量有限的场景。

## 主要策略

| 策略 | 说明 | 适用场景 |
|------|------|----------|
| 特征提取 | 冻结预训练层，只训练新分类头 | 数据量小，任务相似 |
| 微调 | 解冻部分/全部层，低学习率训练 | 数据量适中，需要适配 |
| 从头训练 | 只使用预训练架构 | 数据量大，任务差异大 |

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子
RANDOM_SEED = 42
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"TensorFlow 版本: {tf.__version__}")

## 第一部分：训练基础模型（模型 A）

首先在 Fashion MNIST 上训练一个完整的分类模型，作为迁移学习的源模型。

In [ ]:
# 加载 Fashion MNIST 数据集
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# 数据预处理
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

# 类别名称
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

print(f"训练集: {X_train.shape}")
print(f"验证集: {X_valid.shape}")
print(f"测试集: {X_test.shape}")

In [ ]:
def create_base_model(name='model_A'):
    """
    创建基础分类模型
    
    Returns:
    --------
    keras.Model
        10分类的 Fashion MNIST 模型
    """
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=[28, 28], name='flatten'),
        keras.layers.Dense(300, activation='relu', name='dense_1'),
        keras.layers.Dense(100, activation='relu', name='dense_2'),
        keras.layers.Dense(10, activation='softmax', name='output')
    ], name=name)
    
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        metrics=['accuracy']
    )
    
    return model

# 创建并训练基础模型
model_A = create_base_model()
model_A.summary()

In [ ]:
# 定义回调函数
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    'model_A_best.keras',
    monitor='val_loss',
    save_best_only=True
)

early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=5,
    monitor='val_loss',
    restore_best_weights=True
)

# 训练模型 A
print("训练基础模型 A...")
history_A = model_A.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb, early_stopping_cb],
    verbose=1
)

# 评估
test_loss_A, test_acc_A = model_A.evaluate(X_test, y_test, verbose=0)
print(f"\n模型 A 测试准确率: {test_acc_A:.4f}")

In [ ]:
# 绘制训练曲线
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history_A.history['accuracy'], label='训练')
axes[0].plot(history_A.history['val_accuracy'], label='验证')
axes[0].set_title('模型 A 准确率')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history_A.history['loss'], label='训练')
axes[1].plot(history_A.history['val_loss'], label='验证')
axes[1].set_title('模型 A 损失')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 第二部分：迁移学习（模型 B）

创建新任务：二分类任务（区分上装类 vs 下装类）

- **上装类 (label=1)**: T-shirt, Pullover, Coat, Shirt
- **下装类 (label=0)**: Trouser, Dress, Sandal, Sneaker, Bag, Ankle boot

In [ ]:
# 创建二分类标签
# 上装类: T-shirt(0), Pullover(2), Coat(4), Shirt(6) -> 1
# 其他 -> 0
upper_class_indices = [0, 2, 4, 6]

def convert_to_binary(y):
    """将多分类标签转换为二分类标签"""
    return np.isin(y, upper_class_indices).astype(np.float32)

y_train_B = convert_to_binary(y_train)
y_valid_B = convert_to_binary(y_valid)
y_test_B = convert_to_binary(y_test)

print(f"二分类标签分布:")
print(f"训练集 - 上装: {y_train_B.sum():.0f}, 其他: {len(y_train_B) - y_train_B.sum():.0f}")
print(f"测试集 - 上装: {y_test_B.sum():.0f}, 其他: {len(y_test_B) - y_test_B.sum():.0f}")

### 2.1 特征提取：冻结预训练层

In [ ]:
# 加载预训练模型并克隆
model_A_loaded = keras.models.load_model('model_A_best.keras')

# 克隆模型架构和权重
model_A_clone = keras.models.clone_model(model_A_loaded)
model_A_clone.set_weights(model_A_loaded.get_weights())

print("预训练模型加载完成")
print(f"层列表: {[layer.name for layer in model_A_clone.layers]}")

In [ ]:
# 创建迁移学习模型
# 保留除输出层外的所有层
model_B = keras.models.Sequential(
    model_A_clone.layers[:-1],  # 移除最后的输出层
    name='model_B'
)

# 添加新的二分类输出层
model_B.add(keras.layers.Dense(1, activation='sigmoid', name='binary_output'))

model_B.summary()

In [ ]:
# 第一阶段：冻结预训练层，只训练新的输出层
for layer in model_B.layers[:-1]:
    layer.trainable = False

# 检查可训练参数
print("层的训练状态:")
for layer in model_B.layers:
    print(f"  {layer.name}: trainable={layer.trainable}")

# 使用较小的学习率编译
model_B.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=['accuracy']
)

print(f"\n可训练参数: {model_B.count_params() - sum(layer.count_params() for layer in model_B.layers[:-1])}")

In [ ]:
# 训练第一阶段（特征提取）
print("第一阶段：特征提取（冻结预训练层）...")
history_B_phase1 = model_B.fit(
    X_train, y_train_B,
    epochs=10,
    batch_size=32,
    validation_data=(X_valid, y_valid_B),
    verbose=1
)

# 评估
phase1_acc = model_B.evaluate(X_test, y_test_B, verbose=0)[1]
print(f"\n第一阶段测试准确率: {phase1_acc:.4f}")

### 2.2 微调：解冻部分层

In [ ]:
# 第二阶段：解冻所有层进行微调
for layer in model_B.layers:
    layer.trainable = True

# 使用更小的学习率防止破坏预训练权重
model_B.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.SGD(learning_rate=1e-4),  # 更小的学习率
    metrics=['accuracy']
)

print("层的训练状态（微调）:")
for layer in model_B.layers:
    print(f"  {layer.name}: trainable={layer.trainable}")

In [ ]:
# 训练第二阶段（微调）
print("第二阶段：微调全部层...")

early_stopping_B = keras.callbacks.EarlyStopping(
    patience=5,
    monitor='val_loss',
    restore_best_weights=True
)

history_B_phase2 = model_B.fit(
    X_train, y_train_B,
    epochs=20,
    batch_size=32,
    validation_data=(X_valid, y_valid_B),
    callbacks=[early_stopping_B],
    verbose=1
)

# 最终评估
final_loss, final_acc = model_B.evaluate(X_test, y_test_B, verbose=0)
print(f"\n最终测试准确率: {final_acc:.4f}")

## 第三部分：对比实验

In [ ]:
# 从头训练一个模型进行对比
model_scratch = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
], name='model_scratch')

model_scratch.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=['accuracy']
)

print("训练从头开始的模型...")
history_scratch = model_scratch.fit(
    X_train, y_train_B,
    epochs=20,
    batch_size=32,
    validation_data=(X_valid, y_valid_B),
    callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=0
)

scratch_acc = model_scratch.evaluate(X_test, y_test_B, verbose=0)[1]
print(f"从头训练测试准确率: {scratch_acc:.4f}")

In [ ]:
# 可视化对比结果
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 迁移学习两阶段
all_acc = history_B_phase1.history['val_accuracy'] + history_B_phase2.history['val_accuracy']
all_loss = history_B_phase1.history['val_loss'] + history_B_phase2.history['val_loss']

axes[0].plot(all_acc, 'b-', label='迁移学习', linewidth=2)
axes[0].plot(history_scratch.history['val_accuracy'], 'r--', label='从头训练', linewidth=2)
axes[0].axvline(x=len(history_B_phase1.history['val_accuracy']), color='gray', linestyle=':', label='微调开始')
axes[0].set_title('验证准确率对比')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(all_loss, 'b-', label='迁移学习', linewidth=2)
axes[1].plot(history_scratch.history['val_loss'], 'r--', label='从头训练', linewidth=2)
axes[1].axvline(x=len(history_B_phase1.history['val_loss']), color='gray', linestyle=':', label='微调开始')
axes[1].set_title('验证损失对比')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('transfer_learning_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# 结果汇总
print("\n" + "="*50)
print("结果汇总")
print("="*50)
print(f"迁移学习（微调后）: {final_acc:.4f}")
print(f"从头训练:          {scratch_acc:.4f}")
print(f"提升:              {(final_acc - scratch_acc)*100:+.2f}%")

## 总结：迁移学习最佳实践

### 流程

1. **加载预训练模型**：使用 `clone_model` + `set_weights` 保留权重
2. **替换输出层**：根据新任务修改输出层结构
3. **特征提取阶段**：冻结预训练层，训练新层
4. **微调阶段**：解冻部分/全部层，用小学习率训练

### 关键参数

| 参数 | 特征提取阶段 | 微调阶段 |
|------|-------------|----------|
| 学习率 | 正常 (1e-3) | 较小 (1e-4 ~ 1e-5) |
| 冻结层 | 全部预训练层 | 可选择性冻结底层 |
| Epoch | 较少 (5-10) | 可以更多 |

In [ ]:
# 验证代码正确性
print("迁移学习模块测试完成")
print("\n关键要点:")
print("1. 迁移学习可以利用预训练模型的特征提取能力")
print("2. 分两阶段训练：先冻结后微调")
print("3. 微调时使用更小的学习率保护预训练权重")
print("4. 适合数据量有限但任务相似的场景")